In [1]:
using LinearAlgebra

In [2]:
using Plots
using LaTeXStrings #Latex Syntax for Plotting

In [3]:
using ITensors
# ITensors.compile()

$$ H = -J\sum_{j = 1}^{N - 1}( b^{\dagger}_{j}b_{j+1} + h.c.)+\frac{U}{2} \sum_{j = 1}^{N} n_{j}(n_{j} - 1)$$

**Exact Diagonalization:**

In [4]:
function Create_Annihilation_Creation_Operators(N)
    Creation = zeros(N+1, N+1)
    for i=1:N
        Creation[i+1,i] = sqrt(i)
    end
    Annihilation = Adjoint(Creation)
    return Creation, Annihilation
end

function Enlarge_Matrix_site_j(j, N, matrix)
    # I⊗...⊗I⊗M⊗I...⊗I

    Identity = zeros(size(matrix))
    for i=1:size(matrix)[1]
        Identity[i,i] = 1
    end
    
    M = Identity
    
    if j == 1
        M = matrix
    end    
    
    for i=2:N 
        if i == j
        M = kron(M, matrix)
        else
        M = kron(M, Identity)        
        end
    end

    return M
end

function Enlarge_Matrix_i_Matrix_j(i,j,N, matrix_i, matrix_j)
    # I⊗...⊗I⊗M_i⊗I...⊗I⊗M_j⊗I⊗I...⊗I

    Identity = zeros(size(matrix_i))
    for i=1:size(matrix_i)[1]
        Identity[i,i] = 1
    end

    M = Identity

    
    if j == 1
        M = matrix_j
    end    

    if i == 1
        M = matrix_i
    end   
    
    for k=2:N 
        if k == j
        M = kron(M, matrix_j)
        elseif k == i
        M = kron(M, matrix_i)
        else
        M = kron(M, Identity)        
        end
    end

    return M
end

Enlarge_Matrix_i_Matrix_j (generic function with 1 method)

In [5]:
#Test: 
Creation, Annihilation = Create_Annihilation_Creation_Operators(3)
Identity = zeros(4,4)
for i=1:4
    Identity[i,i] = 1
end
kron(Identity,Creation) == Enlarge_Matrix_site_j(2,2,Creation)
kron(Identity,Creation, Annihilation) == Enlarge_Matrix_i_Matrix_j(2,3,3,Creation, Annihilation)

true

In [6]:
Creation

4×4 Matrix{Float64}:
 0.0  0.0      0.0      0.0
 1.0  0.0      0.0      0.0
 0.0  1.41421  0.0      0.0
 0.0  0.0      1.73205  0.0

In [7]:
function Build_ED_Bose_Hubbard_Hamiltonian(N_Bosons, L_Chain, J, U)
    # H = ∑ J(I⊗...⊗I...⊗I⊗bdag⊗b⊗I⊗...I⊗...⊗I + h.c.) +0.5*U(I⊗...⊗I...⊗I⊗n⊗n⊗I⊗...I⊗...⊗I - ⊗...⊗I⊗n⊗I⊗I...⊗I)

    H = zeros((N_Bosons+1)^L_Chain, (N_Bosons+1)^L_Chain)

    Creation, Annihilation = Create_Annihilation_Creation_Operators(N_Bosons)
    Particle_Number_Op = Creation*Annihilation

    for i=1:L_Chain-1
        j = i + 1 #nearest neighbor in 1D. Let's consider just OBC
        BdagB_Term = Enlarge_Matrix_i_Matrix_j(i,j,L_Chain, Creation, Annihilation)
        BBdag_Term = Enlarge_Matrix_i_Matrix_j(i,j,L_Chain, Annihilation, Creation)
        H = H -J*(BdagB_Term + BBdag_Term)
    end

    for i=1:L_Chain
        nini_Term =  Enlarge_Matrix_site_j(i, L_Chain, Particle_Number_Op*Particle_Number_Op)
        ni_Term = Enlarge_Matrix_site_j(i, L_Chain, Particle_Number_Op)
        H = H + 0.5*U*(nini_Term - ni_Term)
    end    

    return H
end

Build_ED_Bose_Hubbard_Hamiltonian (generic function with 1 method)

In [8]:
H = Build_ED_Bose_Hubbard_Hamiltonian(3, 6, 1, 1)
Eigenvalues = eigen(H).values

4096-element Vector{Float64}:
 -9.48479861742051
 -9.355368957128752
 -9.276847906537574
 -8.930356840283165
 -8.672987764998894
 -8.379190815142868
 -8.337229057949354
 -8.244158663165601
 -8.054206185723405
 -7.9891491546176585
 -7.59079692649008
 -7.398754002944134
 -7.383563887995319
  ⋮
 21.552041784102897
 21.638733732371485
 21.742749315836093
 22.003406729257534
 22.619048826875883
 22.680775179505204
 22.94842339557522
 23.061207708702774
 23.5831113243445
 24.080837270141828
 24.56912961626505
 24.64852700401051

In [9]:
print(Eigenvalues[1], Eigenvalues[2], Eigenvalues[3])

-9.48479861742051-9.355368957128752-9.276847906537574

**Tensor Networks:**

In [10]:
function Build_MPO_Bose_Hubbard_Hamiltonian(N_Bosons, L_Chain, J, U)
    # H = ∑ J(I⊗...⊗I...⊗I⊗bdag⊗b⊗I⊗...I⊗...⊗I + h.c.) +0.5*U(I⊗...⊗I...⊗I⊗n⊗n⊗I⊗...I⊗...⊗I - ⊗...⊗I⊗n⊗I⊗I...⊗I)

    sites = siteinds("Qudit",L_Chain, dim = N_Bosons + 1) #extra parameter: conserve_qns = true  
    # Input the operator terms 
    
    os = OpSum() 
    for i=1:L_Chain-1
        os += -J,"A",i,"Adag",i+1 
        os += -J,"Adag",i,"A",i+1 
    end 

    for i=1:L_Chain
        os += 0.5*U,"N * N",i
        os += -0.5*U,"N",i
    end 
    
    # Convert these terms to an MPO 
    H = MPO(os,sites)
    return H, sites
end

Build_MPO_Bose_Hubbard_Hamiltonian (generic function with 1 method)

In [11]:
H_MPO, sites = Build_MPO_Bose_Hubbard_Hamiltonian(3,6,1,1)

nsweeps = 10
maxdim = [10,20,100,200,200] #maxdim - integer or array of integers specifying the maximum size allowed for the bond dimension or rank of the MPS being optimized
cutoff = [1E-10] #maxdim - integer or array of integers specifying the maximum size allowed for the bond dimension or rank of the MPS being optimized

energy_ground_state, psi_ground_state = dmrg(H_MPO,randomMPS(sites,4); nsweeps, maxdim, cutoff) #randomMPS(sites,4) is the start point. Is a random MPS with bond dimension = number of bosons + 1
energy_1_excited_state, psi_1_excited_state = dmrg(H_MPO,[psi_ground_state], randomMPS(sites,4); nsweeps, maxdim, cutoff) 
energy_2_excited_state, psi_2_excited_state = dmrg(H_MPO,[psi_ground_state, psi_1_excited_state], randomMPS(sites,4); nsweeps, maxdim, cutoff) 

After sweep 1 energy=-9.278177087888508  maxlinkdim=10 maxerr=3.70E-03 time=13.231
After sweep 2 energy=-9.459640110507083  maxlinkdim=20 maxerr=7.00E-07 time=0.047
After sweep 3 energy=-9.475800264208216  maxlinkdim=48 maxerr=9.69E-11 time=0.127
After sweep 4 energy=-9.484354607593012  maxlinkdim=48 maxerr=8.60E-11 time=0.201
After sweep 5 energy=-9.484759213479977  maxlinkdim=43 maxerr=9.40E-11 time=0.175
After sweep 6 energy=-9.4847942315601  maxlinkdim=40 maxerr=9.38E-11 time=0.136
After sweep 7 energy=-9.484798385620225  maxlinkdim=39 maxerr=7.52E-11 time=0.114
After sweep 8 energy=-9.484798610409758  maxlinkdim=38 maxerr=9.64E-11 time=0.127
After sweep 9 energy=-9.484798616215576  maxlinkdim=38 maxerr=7.75E-11 time=0.099
After sweep 10 energy=-9.484798616389247  maxlinkdim=38 maxerr=6.77E-11 time=0.104
After sweep 1 energy=-9.033549826049029  maxlinkdim=10 maxerr=1.42E-03 time=0.212
After sweep 2 energy=-9.266520052346792  maxlinkdim=20 maxerr=1.26E-06 time=0.030
After sweep 3 en

(-9.278159952755809, MPS
[1] ((dim=4|id=413|"Link,l=1"), (dim=4|id=948|"Qudit,Site,n=1"))
[2] ((dim=16|id=154|"Link,l=2"), (dim=4|id=298|"Qudit,Site,n=2"), (dim=4|id=413|"Link,l=1"))
[3] ((dim=4|id=68|"Qudit,Site,n=3"), (dim=36|id=148|"Link,l=3"), (dim=16|id=154|"Link,l=2"))
[4] ((dim=4|id=833|"Qudit,Site,n=4"), (dim=16|id=279|"Link,l=4"), (dim=36|id=148|"Link,l=3"))
[5] ((dim=4|id=913|"Qudit,Site,n=5"), (dim=4|id=463|"Link,l=5"), (dim=16|id=279|"Link,l=4"))
[6] ((dim=4|id=358|"Qudit,Site,n=6"), (dim=4|id=463|"Link,l=5"))
)

In [12]:
# ? randomMPS #Important if we want to include the conservation of nqs

In [13]:
print(Eigenvalues[1], Eigenvalues[2], Eigenvalues[3]) #Using ED

-9.48479861742051-9.355368957128752-9.276847906537574

In [14]:
println(energy_ground_state, energy_1_excited_state, energy_2_excited_state) #Using ITensors

-9.484798616389247-9.353868632235045-9.278159952755809
